<a href="https://colab.research.google.com/github/yajima-yasutoshi/Model/blob/main/20250806/%E7%A2%BA%E8%AA%8D%E3%83%86%E3%82%B9%E3%83%88%E8%A7%A3%E8%AA%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#確認テスト（20250806）

##準備


In [ ]:
%%capture
# python-mip: 数理最適化モデリングライブラリ
!pip install mip

# japanize-matplotlib: Matplotlibでの日本語表示を可能にするライブラリ
!pip install japanize-matplotlib

# 必要なライブラリのインポート
import mip
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
import random
from scipy.spatial.distance import cdist
from itertools import product

-----
##問題1

ある企業が、複数の顧客にサービスを提供するために、いくつかの候補地の中から倉庫（施設）を建設することを計画しています。

* **施設候補地:** 3ヶ所 (F1, F2, F3)。各候補地には倉庫を開設するための固定費用があります。
* **施設候補地の固定費用 ($f_i$):**
    * F1: 固定費用 150
    * F2: 固定費用 100
    * F3: 固定費用 130
* **顧客:** 4ヶ所 (C1, C2, C3, C4)
* **輸送費用 ($c_{ij}$): 施設 $i$ から顧客 $j$ へ輸送する費用:**

|       | C1     | C2       | C3       | C4  |
| :---- | :------: | :------: | :------: | :------: |
| F1  |    20    |    10    |    50    |    30    |
| F2  |    60    |    30    |    20    |    10    |
| F3  |    10    |    40    |    15    |    50    |

どの候補地に施設を開設し、
各顧客へどの開設済みの施設からサービスを提供すれば固定費用と輸送費用の合計が最小になるかを求め、最適値を解答せよ。


In [ ]:
# 1データ定義
# 施設候補地
facilities = ['F1', 'F2', 'F3']
# 変更後の固定費用
fixed_costs = {'F1': 150, 'F2': 100, 'F3': 130}

# 顧客
customers = ['C1', 'C2', 'C3', 'C4']

# 輸送費用
transport_costs = {
    ('F1', 'C1'): 20, ('F1', 'C2'): 10, ('F1', 'C3'): 50, ('F1', 'C4'): 30,
    ('F2', 'C1'): 60, ('F2', 'C2'): 30, ('F2', 'C3'): 20, ('F2', 'C4'): 10,
    ('F3', 'C1'): 10, ('F3', 'C2'): 40, ('F3', 'C3'): 15, ('F3', 'C4'): 50
}

# 2. モデル作成
model = mip.Model("FacilityLocation")

# 3. 決定変数
y = {i: model.add_var(var_type=mip.BINARY, name=f"y_{i}") for i in facilities}
x = {(i, j): model.add_var(var_type=mip.BINARY, name=f"x_{i}_{j}") for i in facilities for j in customers}

# 4. 目的関数: 総費用の最小化
model.objective = mip.minimize(
    mip.xsum(fixed_costs[i] * y[i] for i in facilities) +
    mip.xsum(transport_costs[i, j] * x[i, j] for i in facilities for j in customers)
)

# 5. 制約条件
## 各顧客は必ず1つの施設に割り当てられる
for j_cust in customers:
    model += mip.xsum(x[i_fac, j_cust] for i_fac in facilities) == 1

## 顧客はその施設が開設されている場合にのみ割り当て可能
for i_fac in facilities:
    for j_cust in customers:
        model += x[i_fac, j_cust] <= y[i_fac]


# 6. モデルの最適化と結果表示
status = model.optimize()

if status == mip.OptimizationStatus.OPTIMAL:
    print(f"最小総費用: {model.objective_value:.4f}")
    opened_facilities = [i for i in facilities if y[i].x >= 0.99]
    print(f"開設された施設: {opened_facilities}")
else:
    print("最適解が見つかりませんでした。")

### 問題1-1

開設した施設からサービスを提供できる顧客数を最大3顧客までとした場合、 どの候補地に施設を開設し各顧客へどの開設済みの施設からサービスを提供すれば固定費用と輸送費用の合計が最小になるかを求め、最適値を解答せよ。

In [ ]:
# 1データ定義
# 施設候補地
facilities = ['F1', 'F2', 'F3']
# 変更後の固定費用
fixed_costs = {'F1': 150, 'F2': 100, 'F3': 130}

# 顧客
customers = ['C1', 'C2', 'C3', 'C4']

# 輸送費用
transport_costs = {
    ('F1', 'C1'): 20, ('F1', 'C2'): 10, ('F1', 'C3'): 50, ('F1', 'C4'): 30,
    ('F2', 'C1'): 60, ('F2', 'C2'): 30, ('F2', 'C3'): 20, ('F2', 'C4'): 10,
    ('F3', 'C1'): 10, ('F3', 'C2'): 40, ('F3', 'C3'): 15, ('F3', 'C4'): 50
}

# 2. モデル作成
model = mip.Model("FacilityLocation")

# 3. 決定変数
y = {i: model.add_var(var_type=mip.BINARY, name=f"y_{i}") for i in facilities}
x = {(i, j): model.add_var(var_type=mip.BINARY, name=f"x_{i}_{j}") for i in facilities for j in customers}

# 4. 目的関数: 総費用の最小化
model.objective = mip.minimize(
    mip.xsum(fixed_costs[i] * y[i] for i in facilities) +
    mip.xsum(transport_costs[i, j] * x[i, j] for i in facilities for j in customers)
)

# 5. 制約条件
# 1. 各顧客は必ず1つの施設に割り当てられる
for j_cust in customers:
    model += mip.xsum(x[i_fac, j_cust] for i_fac in facilities) == 1

# 2. 顧客はその施設が開設されている場合にのみ割り当て可能
for i_fac in facilities:
    for j_cust in customers:
        model += x[i_fac, j_cust] <= y[i_fac]

# 3. 1つの施設に割り当てられる顧客は3つまで
for i_fac in facilities:
    model += mip.xsum(x[i_fac, j_cust] for j_cust in customers) <= 3 * y[i_fac]


# 6. モデルの最適化と結果表示
status = model.optimize()

if status == mip.OptimizationStatus.OPTIMAL:
    print(f"最小総費用: {model.objective_value:.4f}")
    opened_facilities = [i for i in facilities if y[i].x >= 0.99]
    print(f"開設された施設: {opened_facilities}")
else:
    print("最適解が見つかりませんでした。")

-----

##問題2

以下のように6都市の座標が与えられたとする。
デポ（出発・帰着）を都市0とし、
6都市全てを巡回する最短経路の総移動距離を求めよ。
移動距離は都市の座標から計算されるユークリッド距離とする。

解答は小点以下第二位を四捨五入して、第一位までを解答する。

###データ

  * **変更後の都市座標**:
      * 都市0: (0, 0)
      * 都市1: (1, 5)
      * 都市2: (4, 3)
      * 都市3: (2, 1)
      * 都市4: (5, 5)
      * 都市5: (2, 5)

#### ユークリッド距離の計算
一般的に、都市Aと都市Bの座標が以下の場合
* 都市A： $(x_A,y_A)$
* 都市B： $(x_B,y_B)$

ユークリッド距離は
$$
\sqrt{(x_A - x_B)^2 + (y_A - y_B)^2}
$$
と計算される。


In [ ]:
import math

# 1. データ定義
cities_coords = {
    0: (0, 0), 1: (1, 5), 2: (4, 3), 3: (2, 1), 4: (5, 5), 5:(2, 5)
}
city_indices = list(cities_coords.keys())
num_cities = len(city_indices)

# 2. 距離行列の計算
costs = {}
for i in city_indices:
    for j in city_indices:
        if i == j:
            continue
        coord1 = cities_coords[i]
        coord2 = cities_coords[j]
        costs[i, j] = math.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)

# 3. モデル作成
model = mip.Model("TSP")

# 4. 決定変数
x = {(i, j): model.add_var(var_type=mip.BINARY, name=f"x_{i}_{j}")
     for i in city_indices for j in city_indices if i != j}
# u_iの範囲は2からN
u = {i: model.add_var(var_type=mip.INTEGER, lb=2, ub=num_cities, name=f"u_{i}")
     for i in city_indices if i != 0}

# 5. 目的関数: 総移動距離の最小化
model.objective = mip.minimize(mip.xsum(costs[i, j] * x[i, j] for i in city_indices for j in city_indices if i != j))

# 6. 制約条件
# 各都市から必ず1つの都市へ出発
for i in city_indices:
    model += mip.xsum(x[i, j] for j in city_indices if i != j) == 1
# 各都市へ必ず1つの都市から到着
for j in city_indices:
    model += mip.xsum(x[i, j] for i in city_indices if i != j) == 1

# MTZ 部分巡回路除去制約
for i in city_indices:
    if i == 0: continue
    for j in city_indices:
        if j == 0 or i == j: continue
        model += u[i] - u[j] + num_cities * x[i, j] <= num_cities - 1

# 7. モデルの最適化と結果表示
status = model.optimize()

if status == mip.OptimizationStatus.OPTIMAL or status == mip.OptimizationStatus.FEASIBLE:
    print(f"最短総移動距離: {model.objective_value:.4f}")
    for i in city_indices:
      if i>0:
        print(f"都市{i}の移動は{u[i].x}番目")
else:
    print("最適解が見つかりませんでした。")


###問題2-1

* 都市1は都市3の前
* 都市3は都市5の前

に訪問しなくてはならないとする。
なお、都市1と都市3の間や
都市3と都市5の間に別の都市が入っても構わない。
この場合の最適値を求めよ。
解答は小点以下第二位を四捨五入して、第一位までを解答する。


In [ ]:
import math

# 1. データ定義
cities_coords = {
    0: (0, 0), 1: (1, 5), 2: (4, 3), 3: (2, 1), 4: (5, 5), 5:(2, 5)
}
city_indices = list(cities_coords.keys())
num_cities = len(city_indices)

# 2. 距離行列の計算
costs = {}
for i in city_indices:
    for j in city_indices:
        if i == j:
            continue
        coord1 = cities_coords[i]
        coord2 = cities_coords[j]
        costs[i, j] = math.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)

# 3. モデル作成
model = mip.Model("TSP")

# 4. 決定変数
x = {(i, j): model.add_var(var_type=mip.BINARY, name=f"x_{i}_{j}")
     for i in city_indices for j in city_indices if i != j}
# u_iの範囲は2からN
u = {i: model.add_var(var_type=mip.INTEGER, lb=2, ub=num_cities, name=f"u_{i}")
     for i in city_indices if i != 0}

# 5. 目的関数: 総移動距離の最小化
model.objective = mip.minimize(mip.xsum(costs[i, j] * x[i, j] for i in city_indices for j in city_indices if i != j))

# 6. 制約条件
# 各都市から必ず1つの都市へ出発
for i in city_indices:
    model += mip.xsum(x[i, j] for j in city_indices if i != j) == 1
# 各都市へ必ず1つの都市から到着
for j in city_indices:
    model += mip.xsum(x[i, j] for i in city_indices if i != j) == 1

# MTZ 部分巡回路除去制約
for i in city_indices:
    if i == 0: continue
    for j in city_indices:
        if j == 0 or i == j: continue
        model += u[i] - u[j] + num_cities * x[i, j] <= num_cities - 1

model += u[3] >= u[1] + 1
model += u[5] >= u[3] + 1

# 7. モデルの最適化と結果表示
status = model.optimize()

if status == mip.OptimizationStatus.OPTIMAL or status == mip.OptimizationStatus.FEASIBLE:
    print(f"最短総移動距離: {model.objective_value:.4f}")
    for i in city_indices:
      if i>0:
        print(f"都市{i}の移動は{u[i].x}番目")
else:
    print("最適解が見つかりませんでした。")

-----

##問題3

ある企業が2つの工場（F1, F2）で製品を生産し、2つの中間倉庫（W1, W2）を経由して、3つの主要な顧客（C1, C2, C3）へ製品を供給する計画を立ている。
各工場には生産上限（供給量）、各顧客には需要量がある。
工場から倉庫へ、倉庫から顧客へ、そして一部の工場から顧客への直接輸送ルートも存在し、それぞれ輸送コストと輸送容量が設定されている。

* **ノードとバランス ($b_k$):**
    * F1 (工場1): 供給量が100 ユニット
    * F2 (工場2): 供給量が80 ユニット
    * W1 (倉庫1): 中継点
    * W2 (倉庫2): 中継点
    * C1 (顧客1): 需要量が85 ユニット (つまり-85ユニットの供給量)
    * C2 (顧客2): 需要量が50 ユニット (つまり-50ユニットの供給量)
    * C3 (顧客3): 需要量が60 ユニット (つまり-60ユニットの供給量)
  
  なお、総供給量 = $100+80=180$、総需要量 = $85+50+60=195$ となり、需要と共有は均衡していない。

* **アーク (始点, 終点, 単位費用, 容量):** 以下の示すようなアークを考える。
    * (F1, W1, 2, 70), (F1, W2, 3, 50)
    * (F2, W1, 4, 60), (F2, W2, 1, 50)
    * (W1, C1, 3, 40), (W1, C2, 5, 30), (W1, C3, 4, 40)
    * (W2, C1, 2, 50), (W2, C2, 1, 40), (W2, C3, 3, 30)
    * (F1, C1, 10, 20)  // F1からC1への直接輸送
    * (F2, C3, 12, 20)  // F2からC3への直接輸送

総需要(195)が総供給(180)を上回り、15単位の需要が満たせない状況である。満たされなかった需要1単位あたり**20**のペナルティ費用が発生する場合、実際の総輸送費用とペナルティ費用の合計を最小化せよ。その最小費用を解答せよ。

#### **解説**

本問題は、総供給と総需要が一致しない「不均衡問題」を扱う応用問題である。実社会では、需要が供給を上回ることは頻繁に発生し、どの顧客への供給を優先（または断念）するかをコストベースで決定する必要がある。

この種の問題は、ネットワークを意図的に「均衡」させることで、標準的な最小費用流問題のソルバーで解くことができる。そのためのテクニックが「ダミーノード」の導入である。

1.  **ダミー供給ノードの導入:**
    需要超過分と等しい供給量を持つ仮想的なノード（ここではDS: Dummy Source）をネットワークに追加する。このノードは、あたかも不足分の商品を供給してくれる供給源のように振る舞う。
    $$b_{DS} = \sum_{k \in \text{Demand}} |b_k| - \sum_{k \in \text{Supply}} b_k = 195 - 180 = 15$$

2.  **ペナルティ付きアークの導入:**
    ダミー供給ノードDSから、全ての実際の需要ノード（C1, C2, C3）に対してアークを追加する。これらのアークの単位輸送費用 $c\_{DS, C\_j}$ を、指定されたペナルティ費用（ここでは20）に設定する。これらのアークをフローが流れるということは、その需要が「ダミー」から供給された、つまり「実際には満たされなかった」ことを意味する。そのフロー量にペナルティ費用が掛けられることで、目的関数上で罰金として計上される。

この拡張されたネットワークでは、総供給と総需要が再び釣り合うため、最小費用流問題を解くことができる。ソルバーは、実際の供給源からの輸送コストと、ダミー供給源からの輸送コスト（＝ペナルティ）を天秤にかけ、全体の費用が最小となるようにフローを決定する。

#### **解答コード**

In [ ]:
# 1. データ定義
# 顧客C1の需要量を変更
balances = {'F1': 100, 'F2': 80, 'W1': 0, 'W2': 0, 'C1': -85, 'C2': -50, 'C3': -60}

# 不足量を計算
total_supply = sum(v for v in balances.values() if v > 0)
total_demand = sum(abs(v) for v in balances.values() if v < 0)
shortage = total_demand - total_supply

# ダミーノードとペナルティ費用を追加
balances['DS'] = shortage  # ダミー供給ノード
node_names = list(balances.keys())
penalty_cost = 20

# 元のアークデータ
arcs_data_original = [
    ('F1', 'W1', 2, 70), ('F1', 'W2', 3, 50), ('F2', 'W1', 4, 60), ('F2', 'W2', 1, 50),
    ('W1', 'C1', 3, 40), ('W1', 'C2', 5, 30), ('W1', 'C3', 4, 40),
    ('W2', 'C1', 2, 50), ('W2', 'C2', 1, 40), ('W2', 'C3', 3, 30),
    ('F1', 'C1', 10, 20), ('F2', 'C3', 12, 20)
]
# ダミーアークを追加
arcs_data_dummy = [
    ('DS', 'C1', penalty_cost, shortage),
    ('DS', 'C2', penalty_cost, shortage),
    ('DS', 'C3', penalty_cost, shortage),
]
arcs_data = arcs_data_original + arcs_data_dummy

defined_arcs = [(u, v) for u, v, cost, cap in arcs_data]
costs = {(u, v): cost for u, v, cost, cap in arcs_data}
capacities = {(u, v): cap for u, v, cost, cap in arcs_data}

# 2. モデル作成
model = mip.Model("MinCostFlow")

# 3. 決定変数
x_flow = {(i, j): model.add_var(name=f"x_{i}_{j}", lb=0, ub=capacities.get((i, j), 0))
          for (i, j) in defined_arcs}

# 4. 目的関数
model.objective = mip.minimize(mip.xsum(costs[i, j] * x_flow[i, j] for (i, j) in defined_arcs))

# 5. 制約条件
for k_node in node_names:
    flow_out = mip.xsum(x_flow[i, j] for (i, j) in defined_arcs if i == k_node)
    flow_in = mip.xsum(x_flow[i, j] for (i, j) in defined_arcs if j == k_node)
    model += flow_out - flow_in == balances[k_node]

# 6. モデルの最適化と結果表示
status = model.optimize()

if status == mip.OptimizationStatus.OPTIMAL:
    print(f"ペナルティ込みでの最小総費用: {model.objective_value:.4f}")
else:
    print("最適解が見つかりませんでした。")


-----

##問題4
ある会社が、製品を特殊なコンテナ（ビン）に詰めて輸送する計画を立てている。各製品（アイテム）はそれぞれ異なる重量と容積を持つ。全ての製品を輸送するために必要な最小のコンテナ数を求めよ。

**データ:**

  * **コンテナの容量:**
   * 最大積載重量 $C\_{weight}$: 17 kg
   * 最大容積 $C\_{volume}$: 11 m³
  * **製品（アイテム）のリスト（重量, 容積）:**
      * アイテム0: (5 kg, 4 m³)
      * アイテム1: (8 kg, 6 m³)
      * アイテム2: (3 kg, 2 m³)
      * アイテム3: (10 kg, 5 m³)
      * アイテム4: (6 kg, 8 m³)
      * アイテム5: (7 kg, 4 m³)
      * アイテム6: (9 kg, 4 m³)
      * アイテム7: (7 kg, 7 m³)

#### **解答コード**

In [ ]:
# 1. 問題データの設定
# コンテナの容量
bin_capacity_weight = 17
bin_capacity_volume = 11

# アイテムのデータ: (重量, 容積)
items_data = [
    (5, 4),  # アイテム0
    (8, 6),  # アイテム1
    (3, 2),  # アイテム2
    (10, 5), # アイテム3
    (6, 8),  # アイテム4
    (7, 4),  # アイテム5
    (9, 4),  # アイテム6
    (7, 7)   # アイテム7

]
item_weights = [item[0] for item in items_data]
item_volumes = [item[1] for item in items_data]
num_items = len(items_data)
num_bins = num_items

# 2. モデルの作成
model = mip.Model(name="2D_bin_packing", sense=mip.MINIMIZE)

# 3. 変数の定義
x = [[model.add_var(var_type=mip.BINARY, name=f"x_{i}_{j}") for j in range(num_bins)] for i in range(num_items)]
y = [model.add_var(var_type=mip.BINARY, name=f"y_{j}") for j in range(num_bins)]

# 4. 目的関数の設定
model.objective = mip.xsum(y[j] for j in range(num_bins))

# 5. 制約条件の追加
# 制約1: 各アイテムは正確に一つのビンに割り当てられる
for i in range(num_items):
    model += mip.xsum(x[i][j] for j in range(num_bins)) == 1

# --- 応用問題で変更された制約 ---
# 制約2a: 各ビンの「重量」容量制約
for j in range(num_bins):
    model += mip.xsum(item_weights[i] * x[i][j] for i in range(num_items)) <= bin_capacity_weight * y[j]

# 制約2b: 各ビンの「容積」容量制約
for j in range(num_bins):
    model += mip.xsum(item_volumes[i] * x[i][j] for i in range(num_items)) <= bin_capacity_volume * y[j]

# 6. 問題の求解と結果の表示
status = model.optimize()

if status == mip.OptimizationStatus.OPTIMAL:
    print(f"2次元制約下での最小コンテナ数: {int(model.objective_value)}")
else:
    print("最適解が見つかりませんでした。")

-----
##問題5

あるネットワークにおいて、ソースノード(S)から
シンクノード(T)へのフローを考える。
アーク容量に加え、一部のノードにはそこを通過するフローに
上限がある。
これらの制約下で、SからTへの最大フローを求めよ。

**問題設定:**

  * **ノード:** S, A, B, C, T
  * **アーク容量:**
      * (S, A): 20
      * (S, B): 15
      * (A, C): 10
      * (B, A): 5
      * (B, C): 12
      * (C, T): 18
      * (A, T): 2
      * (B, T): 2

  * **ノード容量:**
      * ノードA: 18
      * ノードB: 9

**解答コード:**

In [ ]:
import mip

# 1. 問題データ定義
source_node = 'S'
sink_node = 'T'
# 元のネットワーク
arcs_data = [
    ('S', 'A', 20), ('S', 'B', 15),
    ('A', 'C', 10), ('B', 'A', 5),
    ('B', 'C', 12), ('C', 'T', 18),
    ('A', 'T', 2), ('B', 'T', 2)
]
node_capacities = {'A': 18, 'B': 9}

# 2. ノード分割によるネットワーク変換
new_arcs_data = []
# ノード容量を持つノードを分割
for u, v, cap in arcs_data:
    u_new, v_new = u, v
    if u in node_capacities: u_new = u + '_out'
    if v in node_capacities: v_new = v + '_in'
    new_arcs_data.append((u_new, v_new, cap))

# 分割したノード間にアークを追加
for node, cap in node_capacities.items():
    new_arcs_data.append((node + '_in', node + '_out', cap))

# MIP用にデータを整形
defined_arcs = [(u, v) for u, v, cap in new_arcs_data]
capacities = {(u, v): cap for u, v, cap in new_arcs_data}
nodes = set([u for u,v,c in new_arcs_data] + [v for u,v,c in new_arcs_data])

# 3. モデルの作成
model = mip.Model(name="MaxFlow_NodeCapacity", sense=mip.MAXIMIZE)
x_flow = {(i, j): model.add_var(name=f"x_{i}_{j}", lb=0, ub=capacities[i,j]) for (i,j) in defined_arcs}
v_total_flow = model.add_var(name="total_flow", lb=0)
model.objective = v_total_flow

# 4. 制約条件の追加
for k in nodes:
    flow_out = mip.xsum(x_flow[i, j] for (i, j) in defined_arcs if i == k)
    flow_in = mip.xsum(x_flow[i, j] for (i, j) in defined_arcs if j == k)

    if k == source_node:
        model.add_constr(flow_out - flow_in == v_total_flow)
    elif k == sink_node:
        model.add_constr(flow_in - flow_out == v_total_flow)
    else:
        model.add_constr(flow_in == flow_out)

# 5. 求解
status = model.optimize()

# 6. 結果の表示
if status == mip.OptimizationStatus.OPTIMAL:
    print(f"最大フロー量: {model.objective_value:.2f}")
else:
    print("最適解が見つかりませんでした。")

-----

##問題6

ある旅行者が、スタート地点（ノード0）から出発し、制限時間 $T_{max}$ 内にいくつかの観光地（ノード1から5）を巡り、最終的にゴール地点（ノード6）に到達する計画を立てています。各観光地には訪問することで得られるスコアがあります。

* **ノード:**
    * ノード0: スタート地点, 座標(0,0), スコア $p_0=0$
    * ノード1: 観光地A, 座標(1,3), スコア $p_1=10$
    * ノード2: 観光地B, 座標(3,4), スコア $p_2=40$
    * ノード3: 観光地C, 座標(2,3.5), スコア $p_3=10$
    * ノード4: 観光地D, 座標(3,3), スコア $p_2=20$
    * ノード5: 観光地B, 座標(2,4), スコア $p_2=25$
    * ノード6: ゴール地点, 座標(4,1), スコア $p_3=0$
* **移動時間 ($t_{ij}$):** ノード間のユークリッド距離を移動時間とします。
* **総移動時間上限 ($T_{max}$):** 8.2単位時間。

2つの独立した経路 (2人の旅行者、または2台の車両) でスコア地点を分担して訪問できるとします。各経路の総移動時間上限はそれぞれ $T_{max}$ です。
スコア地点は一度訪問されればスコアが得られ、複数の経路で同じスコア地点を訪問してもスコアは重複して加算されません。全体の獲得総スコアの最大値を求めなさい。


In [ ]:
# 必要なライブラリのインポート
from mip import Model, xsum, maximize, BINARY, CONTINUOUS, OptimizationStatus, CBC
import math
import matplotlib.pyplot as plt
import japanize_matplotlib
import numpy as np

# --- データ定義 ---
nodes_data = {
    0: {'name': 'Start', 'coords': (0, 0), 'prize': 0, 'type': 'StartEnd'},
    1: {'name': 'Spot A', 'coords': (1, 3), 'prize': 10, 'type': 'Prize'},
    2: {'name': 'Spot B', 'coords': (3, 4), 'prize': 40, 'type': 'Prize'},
    3: {'name': 'Spot C', 'coords': (2, 3.5), 'prize': 10, 'type': 'Prize'},
    4: {'name': 'Spot D', 'coords': (3, 3), 'prize': 20, 'type': 'Prize'},
    5: {'name': 'Spot E', 'coords': (2, 4), 'prize': 25, 'type': 'Prize'},
    6: {'name': 'End', 'coords': (4, 1), 'prize': 0, 'type': 'StartEnd'}
}
node_indices = list(nodes_data.keys())
start_node = 0
end_node = 6
prize_nodes = [i for i, data in nodes_data.items() if data['type'] == 'Prize']
prizes = {i: nodes_data[i]['prize'] for i in prize_nodes}
num_vehicles = 2 # 経路(旅行者)の数
vehicles = range(num_vehicles)

travel_times = {}
for i in node_indices:
    for j in node_indices:
        if i == j: continue
        coord1 = nodes_data[i]['coords']
        coord2 = nodes_data[j]['coords']
        travel_times[i, j] = math.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)

T_max = 8.2
M = T_max + max(travel_times.values()) + 1

# --- モデル作成 ---
model = Model("Team Orienteering Problem", solver_name=CBC)

# 決定変数
x = {(i, j, p): model.add_var(var_type=BINARY, name=f"x_{i}_{j}_{p}")
     for i in node_indices for j in node_indices if i != j for p in vehicles}
y = {k: model.add_var(var_type=BINARY, name=f"y_{k}") for k in prize_nodes}
u = {(i, p): model.add_var(lb=0, ub=T_max, name=f"u_{i}_{p}") for i in node_indices for p in vehicles}

# 目的関数
model.objective = maximize(xsum(prizes[k] * y[k] for k in prize_nodes))

# --- 制約条件 ---
for p in vehicles:
    # 1. 各経路はSから出発し、Eへ到着
    model += xsum(x[start_node, j, p] for j in node_indices if j != start_node) == 1
    model += xsum(x[i, end_node, p] for i in node_indices if i != end_node) == 1
    # スタートへ戻らない、エンドから出ない
    model += xsum(x[j, start_node, p] for j in node_indices if j != start_node) == 0
    model += xsum(x[end_node, i, p] for i in node_indices if i != end_node) == 0

    # 2. 各経路のフロー保存則
    for k in prize_nodes:
        model += xsum(x[i, k, p] for i in node_indices if i != k) == xsum(x[k, j, p] for j in node_indices if j != k)

    # 4. 各経路の時間累積(MTZ)と時間上限
    model += u[start_node, p] == 0
    for i in node_indices:
        for j in node_indices:
            if i != j and j != start_node:
                model += u[j, p] >= u[i, p] + travel_times[i, j] - M * (1 - x[i, j, p])

# 3. スコア地点訪問制約 (いずれかの経路が訪問)
for k in prize_nodes:
    # 地点kを訪問する経路は最大1つ (y_kで表現)
    model += xsum(x[i, k, p] for i in node_indices if i != k for p in vehicles) == y[k]

# --- 最適化の実行 ---
status = model.optimize(max_seconds=120)

# --- 結果の表示 ---
if status == OptimizationStatus.OPTIMAL or status == OptimizationStatus.FEASIBLE:
    total_prize = model.objective_value
    print(f"最適解が見つかりました。")
    print(f"獲得総スコア: {total_prize:.2f}")

    # --- 結果の図示 ---
    plt.figure(figsize=(10, 8))
    colors = ['blue', 'green']
    # 地点プロット
    visited_nodes = set()
    for p in vehicles:
        for i, j,p in x:
            if x[i, j, p].x > 0.99:
                visited_nodes.add(i)
                visited_nodes.add(j)

    for i, data in nodes_data.items():
        color = 'lightgray'
        if i in visited_nodes:
            color = 'red' if data['type'] == 'Prize' else 'gold'
        plt.scatter(data['coords'][0], data['coords'][1], s=200, color=color, edgecolors='black', zorder=5)
        plt.text(data['coords'][0] + 0.1, data['coords'][1] + 0.1, f"{i}: {data['name']}\n(p={data['prize']})")

    # 経路ごとの結果表示とプロット
    for p in vehicles:
        path = [start_node]
        current_node = start_node
        path_time = 0

        # 経路が何かを訪問しているかチェック
        if sum(x[start_node, j, p].x for j in node_indices if j != start_node) < 0.5:
            print(f"\n経路 {p+1} は使用されませんでした (Start->Endの直行)。")
            continue

        while current_node != end_node:
            for j in node_indices:
                if j != current_node and x[current_node, j, p].x > 0.99:
                    path.append(j)
                    path_time += travel_times[current_node, j]
                    current_node = j
                    break

        print(f"\n経路 {p+1}: {path}")
        print(f"  総移動時間: {u[end_node, p].x:.3f} (上限: {T_max})")

        path_coords = np.array([nodes_data[node_idx]['coords'] for node_idx in path])
        plt.plot(path_coords[:, 0], path_coords[:, 1], '-o', color=colors[p], alpha=0.7, label=f"経路 {p+1}")

    plt.title(f"演習問題4の解\n総スコア: {total_prize:.0f}")
    plt.xlabel("X座標")
    plt.ylabel("Y座標")
    plt.grid(True)
    plt.axis('equal')
    plt.legend()
    plt.show()

else:
    print("解が見つかりませんでした。")